In [2]:
import math
import random

    #Acoustic wave function y = A * exp(-k * x) * sin(B * x)
def model_function(x, A, k, B):
    return A * math.exp(-k * x) * math.sin(B * x)

    """Fitness func 1: Mean Squared Error between data and the parameters (A,k,B).
    Adds weight to large errors. """
def mse(data, params):
    A, k, B = params
    error_sum = 0.0
    for x, y in data:
        y_pred = model_function(x, A, k, B)
        error_sum += (y - y_pred) ** 2
    return error_sum / len(data)

    """Fitness func 2: Mean Absolute Error between data and the parameters (A,k,B).
    Is beneficial for our case because MAE handles all errors and outliers the same."""
def mae(data, params):
    A, k, B = params
    error_sum = 0.0
    for x, y in data:
        y_pred = model_function(x, A, k, B)
        error_sum += abs(y - y_pred)
    return error_sum / len(data)

def simulated_annealing(data, initial_params, fitness_func,
                        max_iters=10000, fitness_thresh=1e-4,
                        initial_temp=1000, cooling_rate=0.995, step_sizes=(0.1, 0.01, 0.1)):
    """
    Simulated Annealing algorithm to minimize the fitness function
    with respect to our parameters (A, k, B).

    Parameters:
    - data: list of tuples(x,y)
    - initial_params: tuple/list of (A,k,B)
    - fitness_func: function(data, params) returning error to minimize
    - max_iters: max iterations to run
    - fitness_thresh: stop if fitness (error) drops below this
    - initial_temp: starting temperature
    - cooling_rate: rate at which temperature decreases per iteration
    - step_sizes: tuple of max changes per param per step (dA, dk, dB)
    """

    current_params = list(initial_params)
    current_fitness = fitness_func(data, current_params)
    best_params = current_params[:]
    best_fitness = current_fitness
    temp = initial_temp

    for iteration in range(max_iters):
        # Generate new candidate params by adding small random perturbation
        candidate_params = []
        for i in range(3):
            change = random.uniform(-step_sizes[i], step_sizes[i])
            candidate_params.append(current_params[i] + change)

        # Calculate fitness of the new candidate
        candidate_fitness = fitness_func(data, candidate_params)

        # Calculate acceptance probability
        if candidate_fitness < current_fitness:
            accept_prob = 1.0
        else:
            # Boltzmann acceptance criterion
            delta = candidate_fitness - current_fitness
            accept_prob = math.exp(-delta / temp)

        # Accept or reject candidate
        if random.random() < accept_prob:
            current_params = candidate_params
            current_fitness = candidate_fitness

            # Update best found
            if current_fitness < best_fitness:
                best_fitness = current_fitness
                best_params = current_params[:]

        # Update temperature
        temp *= cooling_rate

        # Print progress every N iterations
        if iteration % (max_iters // 10) == 0 or best_fitness < fitness_thresh:
            print(f"Iter {iteration}, Best Fitness: {best_fitness:.6f}, Params: {best_params}")

        # Check for stopping
        if best_fitness < fitness_thresh:
            print(f"Reached fitness threshold {fitness_thresh} at iteration {iteration}")
            break

    return best_params, best_fitness


# Example usage with synthetic data and numpy-generated points
if __name__ == "__main__":
    import numpy as np

    # Data generation function (given)
    def generate_points(m_true, b_true, num_points, noise=1.0):
        x = np.linspace(-10, 10, num_points)
        y = m_true * x + b_true + np.random.normal(0, noise, num_points)
        return list(zip(x, y))

    # Instead of linear data, generate data from our model function with noise
    def generate_acoustic_data(A_true, k_true, B_true, num_points=50, noise=0.2):
        x_vals = np.linspace(0, 10, num_points)
        y_vals = [A_true * np.exp(-k_true * x) * np.sin(B_true * x) + random.gauss(0, noise) for x in x_vals]
        return list(zip(x_vals, y_vals))

    # True parameters (for generating data)
    A_true, k_true, B_true = 3.0, 0.3, 2.5
    data = generate_acoustic_data(A_true, k_true, B_true, num_points=100, noise=0.1)

    # Initial guesses for parameters
    initial_params = (1.0, 0.1, 1.0)

    # Run simulated annealing with MSE
    print("Running Simulated Annealing with MSE")
    best_params_mse, best_fitness_mse = simulated_annealing(data, initial_params, mse,
                                                           max_iters=5000, fitness_thresh=1e-5)

    print(f"Best parameters (MSE): {best_params_mse}, Fitness: {best_fitness_mse}")

    # Run simulated annealing with MAE
    print("\nRunning Simulated Annealing with MAE")
    best_params_mae, best_fitness_mae = simulated_annealing(data, initial_params, mae,
                                                           max_iters=5000, fitness_thresh=1e-5)

    print(f"Best parameters (MAE): {best_params_mae}, Fitness: {best_fitness_mae}")

Running Simulated Annealing with MSE loss...
Iter 0, Best Fitness: 0.948257, Params: [1.0, 0.1, 1.0]
Iter 500, Best Fitness: 0.394720, Params: [0.7073753971992568, -0.037272901108084455, 2.5014207827979957]
Iter 1000, Best Fitness: 0.394720, Params: [0.7073753971992568, -0.037272901108084455, 2.5014207827979957]
Iter 1500, Best Fitness: 0.394720, Params: [0.7073753971992568, -0.037272901108084455, 2.5014207827979957]
Iter 2000, Best Fitness: 0.394720, Params: [0.7073753971992568, -0.037272901108084455, 2.5014207827979957]
Iter 2500, Best Fitness: 0.394720, Params: [0.7073753971992568, -0.037272901108084455, 2.5014207827979957]
Iter 3000, Best Fitness: 0.394720, Params: [0.7073753971992568, -0.037272901108084455, 2.5014207827979957]
Iter 3500, Best Fitness: 0.013121, Params: [2.9408006682326646, 0.28135460846345506, 2.489883199657211]
Iter 4000, Best Fitness: 0.011317, Params: [3.1181595065556524, 0.30724397328898595, 2.497961793851734]
Iter 4500, Best Fitness: 0.011301, Params: [3.1144